In [2]:
import traitlets
import ipywidgets.widgets as widgets
# from pygame.time import Clock
from time import sleep
from jetbot import Robot

TICKS_PER_SECOND: int = 30
JOYSTICK_DZ_THRESH: float = 0.15
PRECISION: int = 3


class Collector:
    def __init__(self):
        self.controller = widgets.Controller(index=0)
        
        # Display live feed of controller values
        display(self.controller)
        
        self.robot: Robot = Robot()
        self.controls = {'left_motor': 0.0, 'right_motor': 0.0}
        
        # Dynamically link motor controller values to class attribute
        self.left_link = traitlets.dlink((self.controls['left_motor'], 'value'), (self.robot.left_motor, 'value'))
        self.right_link = traitlets.dlink((self.controls['right_motor'], 'value'), (self.robot.right_motor, 'value'))
#         self.left_link = traitlets.dlink((self.controller.axes[1], 'value'), (self.robot.left_motor, 'value'))
#         self.right_link = traitlets.dlink((self.controller.axes[3], 'value'), (self.robot.right_motor, 'value'))
        
    def shutdown(self):
        self.left_link.unlink()
        self.right_link.unlink()
        self.robot.stop()
    
    def monitor_status(self):
        while True:
            try:
                # Handle left joystick axes positions: right and up are positive (after inverting y axis)
                ls_x = round(self.controller.axes[0], PRECISION)
                ls_y = -round(self.controller.axes[1], PRECISION)
                
                # Compensate for controller deadzone
                if abs(ls_x) < JOYSTICK_DZ_THRESH:
                    ls_x = 0.0
                if abs(ls_y) < JOYSTICK_DZ_THRESH:
                    ls_y = 0.0
                    
                # Calculate intermediaries for tank control conversion
                v = (1 - abs(ls_x)) * (ls_y / 1) + ls_y
                w = (1 - abs(ls_y)) * (ls_x / 1) + ls_x
                
                # Handle right trigger position, normalize value
                right_trigger = round((self.controller.axes[4] + 1.0) / 2.0, PRECISION)
                
                # Translate intermediaries and apply right trigger speed multiplier to compute tank control values
                self.controls['left_motor'] = (v - w) / 2 * right_trigger
                self.controls['right_motor'] = (v + w) / 2 * right_trigger
                
                # Slow loop down to a maximum amount of ticks per second
                #self.clock.tick(TICKS_PER_SECOND)
                time.sleep(0.001)

            except KeyboardInterrupt:
                self.shutdown()
                exit(0)
                
if __name__ == '__main__':
     collector = Collector()
#      collector.monitor_status()

Controller()

TypeError: Each object must be HasTraits, not <class 'float'>

In [5]:
# Cell for manual robot shutdown
left_link.unlink()
right_link.unlink()
robot.stop()